In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import joblib
import os

# 檢查裝置設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [12]:
import pandas as pd
import os

# 1. 讀取主訓練資料 (Processed)
data_path = '../data/processed/youbike_weather_merged.csv'
df = pd.read_csv(data_path)
df['record_time'] = pd.to_datetime(df['record_time'])

# 2. 讀取站點資訊檔 
info_path = '../data/raw/station_info.csv'  

if os.path.exists(info_path):
    print(f" 成功讀取站點資訊檔：{info_path}")
    df_info = pd.read_csv(info_path)
    # 建立快速查詢字典: station_no (轉字串) -> name_tw
    # 去除 "YouBike2.0_" 前綴
    name_map = dict(zip(df_info['station_no'].astype(str), df_info['name_tw'].str.replace('YouBike2.0_', '')))
else:
    print(f"找不到檔案：{info_path}")
    print("將使用備用名稱生成邏輯...")
    name_map = {}

print(f"[Info] 訓練資料總筆數: {len(df)}")

selected_stations = []
station_info_map = {} # 這就是我們要產生的「Dashboard 專用字典」

# 3. 執行分區選站邏輯
if 'district' in df.columns:
    districts = df['district'].unique()
    print(f"發現行政區: {districts}")
    
    for dist in districts:
        # 找出該行政區的所有資料
        dist_df = df[df['district'] == dist]
        
        if not dist_df.empty:
            # 找出該區流量最大者 (出現次數最多者)
            top_station = dist_df['station_no'].value_counts().idxmax()
            selected_stations.append(top_station)
            
            # 查真實中文名字
            # 邏輯：查表 -> 查不到就用 "行政區+代號"
            real_name = name_map.get(str(top_station), f"{dist}熱門站")
            
            # 組合出漂亮的顯示名稱： "捷運科技大樓站 (大安區)"
            station_info_map[str(top_station)] = f"{real_name} ({dist})"
            
    # 取前 5 個不同區域 (你可以改數字，比如取前 8 個也可以)
    top_5_stations = selected_stations[:5]

else:
    print("[Error] 錯誤：找不到 'district' 欄位，無法進行分區篩選")
    top_5_stations = []

print(f" 鎖定 5 個分區代表站點: {top_5_stations}")
print(f" Dashboard 專用字典 :")
print(station_info_map)

# --- 以下標準流程 (建立 Mapping 與 補值) ---
df_filtered = df[df['station_no'].isin(top_5_stations)].copy()

# 製作站點 ID Mapping (0, 1, 2, 3, 4)
station_mapping = {station: idx for idx, station in enumerate(top_5_stations)}
df_filtered['station_idx'] = df_filtered['station_no'].map(station_mapping)

print("Station Mapping created:", station_mapping)

# 補值
features_to_fill = ['bikes_available', 'temperature', 'rain']
df_filtered = df_filtered.sort_values(['station_idx', 'record_time'])
df_filtered[features_to_fill] = df_filtered.groupby('station_idx')[features_to_fill].transform(lambda x: x.ffill().bfill())

print(f"Total samples after filtering: {len(df_filtered)}")

 成功讀取站點資訊檔：../data/raw/station_info.csv
[Info] 訓練資料總筆數: 947940
發現行政區: ['大安區' '大同區' '士林區' '文山區' '中正區' '中山區' '內湖區' '北投區' '松山區' '南港區' '信義區' '萬華區'
 '臺大公館校區']
 鎖定 5 個分區代表站點: [500101001, 500103001, 500104001, 500105001, 500106001]
 Dashboard 專用字典 :
{'500101001': '捷運科技大樓站 (大安區)', '500103001': '延平國宅 (大同區)', '500104001': '劍潭抽水站 (士林區)', '500105001': '台北花木批發市場 (文山區)', '500106001': '臺北自來水事業處 (中正區)', '500107001': '通北街65巷口 (中山區)', '500108001': '文湖街21巷118弄口 (內湖區)', '500109001': '承德路七段304巷口 (北投區)', '500110002': '捷運松山站(4號出口) (松山區)', '500111001': '南港公園(東新街) (南港區)', '500112001': '黎忠區民活動中心 (信義區)', '500113001': '德昌寶興街口(西北角) (萬華區)', '500119005': '臺大水源舍區A棟 (臺大公館校區)'}
Station Mapping created: {500101001: 0, 500103001: 1, 500104001: 2, 500105001: 3, 500106001: 4}
Total samples after filtering: 2775


In [13]:
# 定義特徵欄位
feature_cols = ['bikes_available', 'temperature', 'rain']
target_col = 'bikes_available'

# 1. 建立並訓練 Scaler (只針對數值特徵縮放)
scaler = MinMaxScaler()
df_filtered[feature_cols] = scaler.fit_transform(df_filtered[feature_cols])

# 2. 製作序列資料的函數 (Sliding Window)
def create_multistation_dataset(data, time_steps=3):
    X_list, y_list = [], []
    
    # 針對每一個站點單獨處理，避免時間序列跨越不同站點
    for station_idx in data['station_idx'].unique():
        station_data = data[data['station_idx'] == station_idx]
        
        # 取出數值特徵 (bikes, temp, rain)
        values = station_data[feature_cols].values
        # 取出該站點的 ID
        ids = station_data['station_idx'].values
        
        for i in range(len(values) - time_steps):
            # 準備 Input: 過去 3 小時的資料
            seq_values = values[i:i+time_steps]
            
            # ID 部分: 為了方便模型讀取，將 ID 形狀重塑並接在特徵後方
            seq_ids = ids[i:i+time_steps].reshape(-1, 1)
            
            # 合併: [bikes, temp, rain, station_idx] -> Shape: (3, 4)
            combined_input = np.hstack((seq_values, seq_ids))
            
            # Label: 下一時刻的 bikes_available
            target = values[i + time_steps, 0]
            
            X_list.append(combined_input)
            y_list.append(target)
            
    return np.array(X_list), np.array(y_list)

# 執行資料轉換
TIME_STEPS = 3
X, y = create_multistation_dataset(df_filtered, TIME_STEPS)

# 轉換為 PyTorch Tensor
X_tensor = torch.FloatTensor(X).to(device)
y_tensor = torch.FloatTensor(y).reshape(-1, 1).to(device)

print("Data preparation complete.")
print(f"Input Shape: {X_tensor.shape} (Samples, Time Steps, Features)")

Data preparation complete.
Input Shape: torch.Size([2760, 3, 4]) (Samples, Time Steps, Features)


In [14]:
class MultiStationLSTM(nn.Module):
    def __init__(self, num_stations, input_size=3, hidden_size=64, output_size=1, embedding_dim=5):
        super(MultiStationLSTM, self).__init__()
        
        # 1. 站點嵌入層
        # 輸入: 站點編號索引, 輸出: 站點特徵向量
        self.station_embedding = nn.Embedding(num_stations, embedding_dim)
        
        # 2. LSTM 層
        # 輸入維度 = 數值特徵(3) + 站點嵌入向量(5) = 8
        self.lstm_input_size = input_size + embedding_dim
        self.lstm = nn.LSTM(self.lstm_input_size, hidden_size, batch_first=True)
        
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch, time_steps, 4)
        # 前 3 個是數值特徵，第 4 個是站點 ID
        
        # 拆解輸入
        numerical_features = x[:, :, :3]
        station_ids = x[:, :, 3].long()  # 轉為整數以供 Embedding 使用
        
        # 透過 Embedding 轉換 ID
        station_embedded = self.station_embedding(station_ids)
        
        # 拼接數值特徵與站點特徵
        combined_input = torch.cat((numerical_features, station_embedded), dim=2)
        
        # LSTM 運算
        out, _ = self.lstm(combined_input)
        
        # 取最後時間點輸出
        out = out[:, -1, :] 
        out = self.dropout(out)
        out = self.fc(out)
        return out

# 初始化模型
num_stations = len(station_mapping)
model = MultiStationLSTM(num_stations=num_stations).to(device)

print("Model architecture defined.")
print(model)

Model architecture defined.
MultiStationLSTM(
  (station_embedding): Embedding(5, 5)
  (lstm): LSTM(8, 64, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [15]:
# 設定超參數
learning_rate = 0.001
num_epochs = 100
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 開始訓練
print("Starting training process...")
model.train()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# --- 存檔區 ---
# 確保儲存目錄存在
save_path = '../api/model_files'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 1. 儲存模型權重
torch.save(model.state_dict(), os.path.join(save_path, 'youbike_lstm_multistation.pth'))

# 2. 儲存 Scaler
joblib.dump(scaler, os.path.join(save_path, 'scaler.pkl'))

# 3. 儲存 Station Mapping
joblib.dump(station_mapping, os.path.join(save_path, 'station_mapping.pkl'))

print("Training completed.")
print(f"Files saved to {save_path}:")
print("- youbike_lstm_multistation.pth")
print("- scaler.pkl")
print("- station_mapping.pkl")

Starting training process...
Epoch [10/100], Loss: 0.0655
Epoch [20/100], Loss: 0.0394
Epoch [30/100], Loss: 0.0349
Epoch [40/100], Loss: 0.0316
Epoch [50/100], Loss: 0.0278
Epoch [60/100], Loss: 0.0247
Epoch [70/100], Loss: 0.0200
Epoch [80/100], Loss: 0.0166
Epoch [90/100], Loss: 0.0129
Epoch [100/100], Loss: 0.0112
Training completed.
Files saved to ../api/model_files:
- youbike_lstm_multistation.pth
- scaler.pkl
- station_mapping.pkl
